In [1]:
import pandas as pd
import plotly.express as px
import json
import plotly.graph_objects as go
import geopandas as gpd


data = pd.read_stata('C:/Users/apillai/OneDrive - World Justice Project/Documents/GitHub/EU-NUTS-GIS/Data/TrialData.dta')

recode_values = {
    'A lot': 1, 'Some': 1,
    'A little': 0, 'No trust': 0, 'Don\'t know': 0, 'No answer': 0
}

for column in ['TRT_police', 'TRT_prosecutors', 'TRT_pda', 'TRT_judges']:
    data[column] = data[column].map(recode_values)

long_format = data.melt(id_vars=['nuts_id','country_name_ltn', 'nuts_ltn'], value_vars=['TRT_police', 'TRT_prosecutors', 'TRT_pda', 'TRT_judges'],
                        var_name='Variable', value_name='Value')

long_format_clean = long_format[long_format['nuts_id'].notnull() & (long_format['nuts_id'].str.strip() != '')]

percentage_trust = long_format_clean.groupby(['nuts_id', 'Variable','country_name_ltn', 'nuts_ltn'])['Value'].mean() * 100
percentage_trust_df = percentage_trust.reset_index()
percentage_trust_df.rename(columns={'Value': 'Percentage'}, inplace=True)

def categorize_percentage(value):
    if value <= 10:
        return "0%-10%"
    elif 10 < value <= 25:
        return "10%-25%"
    elif 25 < value <= 50:
        return "25%-50%"
    elif 50 < value <= 75:
        return "50%-75%"
    elif 75 < value <= 90:
        return "75%-90%"
    else:
        return "90%-100%"


percentage_trust_df['cat4map'] = percentage_trust_df['Percentage'].apply(categorize_percentage)


gdf = gpd.read_file("C:/Users/apillai/OneDrive - World Justice Project/Documents/GitHub/EU-NUTS-GIS/Data/EU_base_map.geojson")


data4map = percentage_trust_df.merge(gdf, how='left', left_on='nuts_id', right_on='polID') 



In [3]:

# from dash import Dash, dcc, html
# from base64 import b64encode
# import io

# buffer = io.StringIO()
# html_bytes = buffer.getvalue().encode()
# encoded = b64encode(html_bytes).decode()

# app = Dash()
# app.layout = html.Div([
#     dcc.Graph(figure=fig),
#     html.A(
#         html.Button("Download as HTML"), 
#         id="download",
#         href="data:text/html;base64," + encoded,
#         download="plotly_graph.html"
#     )
# ])

# app.run_server(debug=True, use_reloader=False)  

Original Dumbell

In [8]:
# import plotly.graph_objects as go


# fig = go.Figure()


# countries = data4map['country_name_ltn'].unique()
# variables = data4map['Variable'].unique()

# color_sequence = px.colors.qualitative.Set1  

# country_colors = {country: color_sequence[i % len(color_sequence)] for i, country in enumerate(countries)}

# for country in countries:
#     for variable in variables:
        
#         filtered_data = data4map[(data4map['country_name_ltn'] == country) & 
#                                  (data4map['Variable'] == variable)]
#         fig.add_trace(go.Scatter(
#             x=filtered_data['Percentage'],
#             y=[country] * len(filtered_data),  
#             mode='lines+markers',
#             name=country,
#             legendgroup=country,  
#             visible=(variable == variables[0]),  
#             line=dict(color=country_colors[country]),
            
#         ))


# buttons = []
# for variable in variables:

#     visibility = [(data4map['Variable'] == variable).values.tolist()]

#     visibility = [item for sublist in visibility for item in sublist]

#     buttons.append(
#         dict(
#             label=variable,
#             method='update',
#             args=[{'visible': visibility},
#                   {'title': f'Dumbbell Chart for {variable}'}]
#         )
#     )


# fig.update_layout(
#     updatemenus=[{
#         'buttons': buttons,
#         'direction': 'down',
#         'showactive': True,
#         'x': 0.1,
#         'xanchor': 'left',
#         'y': 1.15,
#         'yanchor': 'top'
#     }],
#     title=f'Dumbbell Chart for {variables[0]}',
#     xaxis=dict(title='Percentage'),
#     yaxis=dict(title='Country'),
#     legend=dict(title='Country', traceorder='normal')
# )


# fig.show()

# fig.write_html('C:/Users/apillai/OneDrive - World Justice Project/Documents/GitHub/EU-NUTS-GIS/Data/ColorDumbbell2.html')

In [36]:
import plotly.graph_objects as go


fig = go.Figure()


countries = data4map['country_name_ltn'].unique()
variables = data4map['Variable'].unique()

color_sequence = px.colors.qualitative.Set1  

country_colors = {country: color_sequence[i % len(color_sequence)] for i, country in enumerate(countries)}

for country in countries:
    for variable in variables:
        
        filtered_data = data4map[(data4map['country_name_ltn'] == country) & 
                                 (data4map['Variable'] == variable)]
        fig.add_trace(go.Scatter(
            x=filtered_data['Percentage'],
            y=[country] * len(filtered_data),  
            mode='lines+markers',
            name=country,
            legendgroup=country,  
            visible=(variable == variables[0]),  
            line=dict(color=country_colors[country]),
            
        ))


buttons = []
for variable in variables:

    visibility = [(data4map['Variable'] == variable).values.tolist()]

    visibility = [item for sublist in visibility for item in sublist]

    buttons.append(
        dict(
            label=variable,
            method='update',
            args=[{'visible': visibility},
                  {'title': f'Dumbbell Chart for {variable}'}]
        )
    )


fig.update_layout(
    margin=dict(l=20, r=0, t=50, b=100),  
    legend=dict(
        title='Country',
        traceorder='normal',
        bgcolor='rgba(255,255,255,0.5)',
        x=1,  
        xanchor='left',
        y=1,
        yanchor='top'
    ),
    
    updatemenus=[{
        'buttons': buttons,
        'direction': 'down',
        'showactive': True,
        'x': 1.25,  
        'xanchor': 'left',
        'y': 1,
        'yanchor': 'top'
    }],
    title=f'Dumbbell Chart for Trust',
    xaxis=dict(title='Percentage'),
    yaxis=dict(title='Country'),
)



fig.show()

fig.write_html('C:/Users/apillai/OneDrive - World Justice Project/Documents/GitHub/EU-NUTS-GIS/Data/Dumbbell.html')